In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils import (
    read_single_problem_from_path_as_adjacency,
    check_validity_for_adjacency
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, OTHER_INSTANCES_BEST_KNOWN, BASE_INSTANCES_BEST_KNOWN
from heuristics import descending_degree_glutonous_heuristic
from meta_heuristics import base_vns_meta_heuristic

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "brock200_2.col"
)
starting_clique=descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
clique = base_vns_meta_heuristic(starting_clique=starting_clique, graph=graph, degrees=degrees, max_time=30)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))
print("Is the clique different ?", all(clique== starting_clique))


Elapsed time 0.03099966049194336
Old 7 New 9 Iteration 1
Elapsed time 0.11203455924987793
Old 9 New 10 Iteration 4
Is the clique valid ?  True


In [ ]:

print(np.sum(clique))

False
9
